## Apiux & SII: Analisis exploratorio de datos y depuracion de data sociedades
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


## Carga de relaciones societarias y depuracion de data

In [3]:
Carpeta = "Proceso Renta 2025/"
Archivo = "JBA_SOC_2024.csv"
Carpeta_Salida = "Renta 2025/"

In [4]:
PATH = "../../"
csv_path = PATH+"data/raw/"+Carpeta+Archivo
vo_pd = pd.read_csv(csv_path, delimiter=';')

In [5]:
vo_pd

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,PERI_AGNO_MES_TRIBUTARIO,FUENTE
0,318500215,12,100.00,100.00,202400,RIAC
1,319519583,12,100.00,100.00,202400,RIAC
2,350003575,29485615,100.00,100.00,202400,F1948
3,350004975,31161877,33.33,33.33,202400,RIAC
4,350004975,53717298,33.33,33.33,202400,RIAC
...,...,...,...,...,...,...
3782060,697198885,697168365,80.00,80.00,202400,RIAC
3782061,697198955,53430011,99.90,99.90,202400,RIAC
3782062,697198955,53430046,0.10,0.10,202400,RIAC
3782063,697199025,75474502,1.00,1.00,202400,F1948


In [6]:
vo_spark = spark.read.option("header", "true").option("delimiter", ";").csv(csv_path)

In [7]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from libsdf.jab_soc_2023_inom where PORCENTAJE_CAPITAL is null or PORCENTAJE_UTILIDADES IS NULL").show()

Hive Session ID = 10ebdae1-1135-44ad-90e2-52a4d83774b3


+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



Primero, veamos los valores null en participacion de capital y participacion de utilidades.

In [8]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from libsdf.jab_soc_2023_inom where PORCENTAJE_CAPITAL is null or PORCENTAJE_UTILIDADES IS NULL").show()

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



Vemos que no hay valores nulos en la tabla. A continuacion veamos cuantos duplicados existen en las columnas de interes:
RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, pues todos
los calculos lo haremos basados en esta columnas.

In [9]:
spark.sql("select * from libsdf.jab_soc_2023_inom where RUT_SOCIEDAD like 'dwElNqcCQQiFyI3ic%' ").show()

[Stage 9:=================================================>         (5 + 1) / 6]

+--------------------+--------------------+------------------+---------------------+------------------------+------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+--------------------+--------------------+------------------+---------------------+------------------------+------+
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|  

In [10]:
vo_spark.createOrReplaceTempView("sociedad")

In [11]:
spark.sql("select count(RUT_SOCIEDAD) from sociedad").show()

[Stage 11:======================================>                  (8 + 4) / 12]

+-------------------+
|count(RUT_SOCIEDAD)|
+-------------------+
|            3782065|
+-------------------+



Donde seleccionamos los valores no repetidos. Haciendo nuevamente un recuento de los valores unicos,

In [12]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, count(*) as count from sociedad group by RUT_SOCIEDAD, RUT_SOCIO order by count desc ").show()

[Stage 14:=====================================>               (143 + 12) / 200]

+------------+---------+-----+
|RUT_SOCIEDAD|RUT_SOCIO|count|
+------------+---------+-----+
|   350634625| 70225601|    1|
|   351113565|532269134|    1|
|   352200035| 73004888|    1|
|   352231885| 20283268|    1|
|   352537365| 51578392|    1|
|   352679605| 48287496|    1|
|   352721325| 49372251|    1|
|   353312055| 26067634|    1|
|   354577725| 52187420|    1|
|   354729205| 63525558|    1|
|   354844425| 69721258|    1|
|   354893565| 47366520|    1|
|   355244965| 42422217|    1|
|   355388605| 32157739|    1|
|   355422625| 72866610|    1|
|   355573335| 49626176|    1|
|   355588455| 71558688|    1|
|   355602735| 44969741|    1|
|   355648655| 53037570|    1|
|   355710885| 44749374|    1|
+------------+---------+-----+
only showing top 20 rows



Por lo visto tenemos unicidad de la relaciones sociedad socio, a juzgar por el recuento de las combinaciones. Ahora veremos cuanto suman los valores de PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES para cada una de las sociedades.


In [13]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL DESC").show()

[Stage 16:================================>                    (124 + 15) / 200]

+------------+-------+----------+
|RUT_SOCIEDAD|CAPITAL|UTILIDADES|
+------------+-------+----------+
|   543658015|  110.0|     110.0|
|   532364915|  110.0|     110.0|
|   552446375|  110.0|     100.0|
|   532240266|  110.0|     100.0|
|   532517669|  110.0|     100.0|
|   534301045|  110.0|     100.0|
|   532244361|  110.0|     110.0|
|   536608945|  110.0|     110.0|
|   678090705|  110.0|     110.0|
|   532893219|  110.0|     100.0|
|   535973254|  110.0|     110.0|
|   532954707|  110.0|     100.0|
|   552253805|  110.0|     110.0|
|   544924595|  110.0|     110.0|
|   536115445|  110.0|     110.0|
|   534660859|  110.0|     110.0|
|   534641406|  110.0|     110.0|
|   373141858|  110.0|     110.0|
|   533085516|  110.0|     100.0|
|   542936525|  110.0|     110.0|
+------------+-------+----------+
only showing top 20 rows



In [14]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL ASC").show()

[Stage 18:============================================>        (169 + 12) / 200]

+------------+------------------+------------------+
|RUT_SOCIEDAD|           CAPITAL|        UTILIDADES|
+------------+------------------+------------------+
|   580152305|10.999999999999892|10.999999999999892|
|   413676555|              90.0|              90.0|
|   540147928|              90.0|              90.0|
|   455299773|              90.0|              90.0|
|   542001598|              90.0|              90.0|
|   536094900|              90.0|              90.0|
|   544612850|              90.0|              90.0|
|   558565565|              90.0|              90.0|
|   532256954|              90.0|              90.0|
|   544098385|              90.0|              90.0|
|   532494569|              90.0|              50.0|
|   533667153|              90.0|              90.0|
|   538051953|              90.0|              90.0|
|   535745075|              90.0|             100.0|
|   538416422|              90.0|              90.0|
|   373217913|              90.0|             

In [15]:
anomalias_sociedades=spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD having CAPITAL>105 or CAPITAL<95").toPandas()
anomalias_sociedades.to_csv(PATH+f'data/processed/{Carpeta_Salida}anomalias_capital_sociedades.csv', index=False)
print(anomalias_sociedades.describe())

          CAPITAL  UTILIDADES
count  375.000000  375.000000
mean    96.033493   95.749280
std      9.255714    9.339595
min     11.000000   11.000000
25%     90.000000   90.000000
50%     91.740000   92.280000
75%    106.610000  100.000000
max    110.000000  127.000000


Ahora vamos las entradas con al menos un valor cero (que indica cero participacion porcentual)

In [16]:
spark.sql("select COUNT(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 OR PORCENTAJE_UTILIDADES=0").show()

[Stage 21:=====================================================>  (39 + 2) / 41]

+--------+
|count(1)|
+--------+
| 1254988|
+--------+



Por lo visto, tenemos 1057765 entradas donde al menos uno de ambos porcentajes es cero. Por otro lado, tenemos 
1053936 registros donde ambos porcentajes son cero.


Ahora veamos cuales porcentajes de capital son cero y luego los porcentajes de utilidades son cero.

In [17]:
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 and PORCENTAJE_UTILIDADES!=0").show()
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL!=0 and PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
|    4458|
+--------+



[Stage 25:==================================================>     (37 + 4) / 41]

+--------+
|count(1)|
+--------+
|    1325|
+--------+



Para el analisis del problema de oscuridad, es mejor tener en cuenta los porcentajes de participacion de capital, porque los creditos se reparten segun la participacion societaria.
Ahora veamos cuantos tienen valores positivos mayores que 100 o negativos.

In [18]:
spark.sql("select * from sociedad WHERE PORCENTAJE_CAPITAL<0 or PORCENTAJE_CAPITAL>100 or PORCENTAJE_UTILIDADES<0 or PORCENTAJE_UTILIDADES>100").show()

+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
+------------+---------+------------------+---------------------+------------------------+------+



El cual es solo un valor levemente superior a 100 %. Seleccionamos los que no tienen valores cero en PORCENTAJE_CAPITAL.
IMPORTANTE: de ser utilizado PORCENTAJE_UTILIDADES se debe filtrar sobre esa columna.

In [19]:
spark.sql("select * from sociedad where PORCENTAJE_CAPITAL!=0").createOrReplaceTempView("sociedad")

In [20]:
spark.sql("select count(*) from sociedad").show()

[Stage 31:=====================================================>  (39 + 2) / 41]

+--------+
|count(1)|
+--------+
| 2528402|
+--------+



In [21]:
spark.sql("select count(*) from sociedad where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()
spark.sql("select *  from libsdf.jab_soc_2023_inom where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



[Stage 38:===================================================>    (11 + 1) / 12]

+--------------------+--------------------+------------------+---------------------+------------------------+------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+--------------------+--------------------+------------------+---------------------+------------------------+------+
|Qbau/6SlJ/lEcKUDF...|RQ0TnfBpncysCFzap...|              0.03|                 0.03|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|i8odveNc3AEZRZWI7...|              0.01|                 0.01|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|9cDQjPbdfB1EfPUmw...|              0.02|                 0.02|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|7T3+u8inor8kIrUtm...|              0.02|                 0.02|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|CbAnOeVKkXKx8w0q3...|              0.01|                 0.01|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|dEi5RAN/WiSIYuK1s...|              0.01|  

In [22]:
sociedad=spark.sql("select * from sociedad").toPandas()
sociedad.to_csv(PATH+f'data/processed/{Carpeta_Salida}sociedades_participacion_capital_nozero.csv', index=False)

In [23]:
sociedad

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,PERI_AGNO_MES_TRIBUTARIO,FUENTE
0,318500215,12,100.00,100.00,202400,RIAC
1,319519583,12,100.00,100.00,202400,RIAC
2,350003575,29485615,100.00,100.00,202400,F1948
3,350004975,31161877,33.33,33.33,202400,RIAC
4,350004975,53717298,33.33,33.33,202400,RIAC
...,...,...,...,...,...,...
2528397,697198885,75038731,20.00,20.00,202400,RIAC
2528398,697198885,697168365,80.00,80.00,202400,RIAC
2528399,697198955,53430011,99.90,99.90,202400,RIAC
2528400,697199025,75474502,1.00,1.00,202400,F1948
